In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").getOrCreate()

In [ ]:
! git clone  https://gitlab.com/adilkarimscience/pysparktp.git

In [ ]:
! ls pysparktp

In [ ]:
df = spark.read.csv('C:/Users/Lenovo T440/pysparktp/diabetes.csv',header=True,inferSchema=True)

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
print(df.count(),len(df.columns))

In [ ]:
df.groupby('Outcome').count().show()

In [ ]:
df.describe().show()

In [ ]:
# Find for null values
for col in df.columns:
  print(col +":",df[df[col].isNull()].count())

In [ ]:
# find total number of 0 values in columns : Glucose, BloodPressure, SkinThikness ,Insulin ,BMI
def count_zeros():
  columns_list = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
  for i in columns_list:
    print(i+":",df[df[i]==0].count())

In [ ]:
count_zeros()

In [ ]:
for i in df.columns[1:6]:
  data = df.agg({i:'mean'}).first()[0]
  print("Mean value for {} is {}".format(i,int(data)))

In [ ]:
from pyspark.sql.functions import *
for i in df.columns[1:6]:
  data = df.agg({i:'mean'}).first()[0]
  print("Mean value for {} is {}".format(i,int(data)))
  df = df.withColumn(i,when(df[i]==0, int(data)).otherwise(df[i]))

In [ ]:
df.show()

In [ ]:
for col in df.columns:
  print("correlation to outcome for {} is {}".format(col,df.stat.corr('Outcome',col)))

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'],outputCol='features')
output_data = assembler.transform(df)

In [ ]:
output_data.printSchema()

In [ ]:
output_data.show()

In [ ]:
from pyspark.ml.classification import LogisticRegression
final_data = output_data.select('features','Outcome')

In [ ]:
final_data.printSchema()

In [ ]:
train, test = final_data.randomSplit([0.7,0.3])
# Create our logistic Regression model
models =  LogisticRegression(labelCol='Outcome')

model = models.fit(train)


In [ ]:
summary = model.summary

In [ ]:
summary.predictions.describe().show()

Evaluation test model

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.evaluate(test)

In [ ]:
predictions.predictions.show(20)

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='Outcome')
evaluator.evaluate(model.transform(test))